In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import json

In [ ]:
data = Path('./data/board.json')
with open(data, 'r') as f:
    data = json.load(f)

In [ ]:
df = pd.DataFrame(data['cards'])
lists = {_['id']: _['name'] for _ in data['lists']}
df['list'] = df['idList'].map(lists)

members = {_['id']: _['username'] for _ in data['members']}
df['members'] = df['idMembers'].apply(lambda x: [members[_] for _ in x])

labels = {_['id']: _['name'] for _ in data['labels']}
df['labels'] = df['idLabels'].apply(lambda x: [labels[_] for _ in x])

In [ ]:
cost_per_point = 500
df_output = df[df['list'] == 'App V5'].copy()
df_output['points'] = df_output['pluginData'].apply(lambda x: json.loads(x[0].get('value', {})).get('points', np.nan))
df_output = df_output[['name', 'desc', 'points', 'members', 'labels']]
df_output['cost'] = df_output['points'] * cost_per_point
df_output['sprint'] = df_output['labels'].apply(lambda x: [y for y in x if 'Sprint' in y][0])
df_output['Asheck-able'] = df_output['members'].apply(lambda x: 'nyashaasheckhove' in x).map({True: 'Yes', False: 'No'})
df_output.drop(['members', 'labels'], axis=1, inplace=True)
df_output.columns = [c.capitalize() for c in df_output.columns]
df_output

In [ ]:
df_output.groupby('Asheck-able').sum()

In [ ]:
df_output.groupby('Sprint').sum()

In [ ]:
df_output.to_csv('output.csv', index=False)